# Retail Analytics & Churn Prediction (SQL + Excel + Tableau + ML)

Run this notebook to train churn models and export `customer_scored.csv` for Tableau.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, roc_auc_score, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
DATA_DIR = Path('..') / 'data'
features = pd.read_csv(DATA_DIR / 'customer_features.csv', parse_dates=['signup_date','last_purchase_date'])
features.head()

## Prepare Features & Target

In [ ]:
use_cols = ['region','channel','segment','age','tenure_days','recency_days','frequency','monetary','avg_order_value','churned']
df = features[use_cols].copy()
X = df.drop('churned', axis=1)
y = df['churned'].astype(int)
num_cols = ['age','tenure_days','recency_days','frequency','monetary','avg_order_value']
cat_cols = ['region','channel','segment']
numeric_tf = Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())])
categorical_tf = Pipeline([('impute', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocess = ColumnTransformer([('num', numeric_tf, num_cols), ('cat', categorical_tf, cat_cols)])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train.shape, X_test.shape

## Logistic Regression

In [ ]:
lr = Pipeline([('preprocess', preprocess), ('model', LogisticRegression(max_iter=1000))])
lr.fit(X_train, y_train)
proba_lr = lr.predict_proba(X_test)[:,1]
pred_lr = (proba_lr>=0.5).astype(int)
print(classification_report(y_test, pred_lr))
print('ROC AUC:', roc_auc_score(y_test, proba_lr))
RocCurveDisplay.from_predictions(y_test, proba_lr)
plt.title('Logistic Regression ROC')
plt.show()

## Random Forest

In [ ]:
rf = Pipeline([('preprocess', preprocess), ('model', RandomForestClassifier(n_estimators=300, max_depth=None, min_samples_split=4, min_samples_leaf=2, random_state=42))])
rf.fit(X_train, y_train)
proba_rf = rf.predict_proba(X_test)[:,1]
pred_rf = (proba_rf>=0.5).astype(int)
print(classification_report(y_test, pred_rf))
print('ROC AUC:', roc_auc_score(y_test, proba_rf))
RocCurveDisplay.from_predictions(y_test, proba_rf)
plt.title('Random Forest ROC')
plt.show()

## Feature Importance

In [ ]:
ohe = rf.named_steps['preprocess'].named_transformers_['cat'].named_steps['onehot']
cat_names = ohe.get_feature_names_out(categorical_features=['region','channel','segment'])
feature_names = np.r_[['age','tenure_days','recency_days','frequency','monetary','avg_order_value'], cat_names]
importances = rf.named_steps['model'].feature_importances_
imp = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)
imp.head(20)
plt.figure()
plt.barh(imp['feature'][:20][::-1], imp['importance'][:20][::-1])
plt.title('Top 20 Features')
plt.tight_layout()
plt.show()

## Export scored dataset for Tableau

In [ ]:
from sklearn.metrics import roc_auc_score
auc_lr = roc_auc_score(y_test, proba_lr)
auc_rf = roc_auc_score(y_test, proba_rf)
best = rf if auc_rf >= auc_lr else lr
proba_all = best.predict_proba(X)[:,1]
scored = features.assign(churn_probability=proba_all)
scored.to_csv(DATA_DIR / 'customer_scored.csv', index=False)
scored.head()